In [1]:
import igl
import scipy as sp
import meshplot as mp
import numpy as np
from meshplot import plot, subplot, interact
import math
import random as rnd

In [2]:
v, f = igl.read_triangle_mesh("data/30_original.off")
f.shape, type(f), v.shape, type(v)

((866694, 3), numpy.ndarray, (508963, 3), numpy.ndarray)

In [48]:
TH_RATIO_CRV = 0.9
TH_COSINE = 0.75

In [3]:
#Compute facet normals
n_faces = igl.per_face_normals(v, f, np.array([0.0, 0.0, 0.0]))
n_vertex = igl.per_vertex_normals(v, f)
k = igl.gaussian_curvature(v, f)
#v1, v2: principal directions | k1, k2: principal values
#v1, v2, k1, k2 = igl.principal_curvature(v, f, 2, True)

alst = igl.adjacency_list(f)
VF,VFi = igl.vertex_triangle_adjacency(f, len(v))

In [4]:
def calculate_principal(ivx):
    ltf = []
    lvt = []
    lvt.append(ivx)
    #print("*", ivx, alst[ivx])

    for fx in VF[VFi[ivx]:VFi[ivx+1]]:
        #print(fx, f[fx])
        ltf.append(fx)

    for vr1 in alst[ivx]:
        if vr1 not in lvt:
            lvt.append(vr1)

        #print("*", vr1, alst[vr1])

        for fx in VF[VFi[vr1]:VFi[vr1+1]]:
            #print(fx, f[fx])
            if fx not in ltf:
                ltf.append(fx)

        for vr2 in alst[vr1]:
            if vr2 not in lvt:
                lvt.append(vr2)
                
    vx_dtnr = {}
    for ix, vtx in enumerate(lvt):
        vx_dtnr[vtx] = ix

    vc = np.array([v[lvt[i]] for i in range(len(lvt))])
    fc = np.array([[vx_dtnr[f[ltf[i]][0]], vx_dtnr[f[ltf[i]][1]], vx_dtnr[f[ltf[i]][2]]] for i in range(len(ltf))], dtype=int)
    v1c, v2c, k1c, k2c = igl.principal_curvature(vc, fc)
    return k1c[0], k2c[0]

In [17]:
#Calculate curvatures for sampled points
area_t = 0.5 * sum(igl.doublearea(v, f))

v_size = min(round(area_t/4), round(v.shape[0] / 2))
v_ratio = math.floor(v.shape[0] / v_size)
v_size = math.floor(v.shape[0] / v_ratio)
v_ratio, v_size

(50, 10179)

In [20]:
from datetime import datetime
start = datetime.now()

#pk1 = np.percentile(abs(k1), 10)
#pk2 = np.percentile(abs(k2), 10)
v_indices = [] #np.array([0 for i in range(v_size)])
k1_c = []
k2_c = []

from datetime import datetime
start = datetime.now()

for i in range(0, v_size):
    while True:
        index = rnd.randint(v_ratio * i, min(v_ratio * (i + 1) - 1, v.shape[0] - 1))
        if len(alst[index]) > 0:
            break

    #if abs(k1[index]) > pk1 or abs(k2[index]) > pk2:
    v_indices.append(index) #v_indices[i] = index;
    k1c, k2c = calculate_principal(index)
    #print(index, k1c, k2c)
    k1_c.append(k1c)
    k2_c.append(k2c)

end = datetime.now()
print(end - start)

0:00:06.897946


In [29]:
np.round(np.percentile(abs(k1_c), 15), 6), np.round(np.percentile(abs(k1_c), 50), 6), np.round(np.percentile(abs(k1_c), 85), 6)

(0.006657, 0.024947, 0.065253)

In [30]:
np.round(np.percentile(abs(k2_c), 15), 6), np.round(np.percentile(abs(k2_c), 50), 6), np.round(np.percentile(abs(k2_c), 85), 6)

(0.009673, 0.035061, 0.123569)

In [43]:
def projectPointPlane(point, plane, planeD):
    k = point.dot(plane) + planeD
    return point - k * plane

In [42]:
pk1 = np.percentile(abs(k1_c), 50)
pk2 = np.percentile(abs(k2_c), 50)
c_idx = []
s_idx = []

for i in range(0, v_size):
    if abs(k1_c[i]) > pk1 and abs(k2_c[i]) > pk2:
        c_idx.append(i)
        s_idx.append(v_indices[i])

In [52]:
# iPoints : indexes list of points Sampled
def baseReflection(iPoints):
    sample_size = len(iPoints)
    lxPairs = []
    
    for i in range(0, sample_size - 1):
        for j in range(i + 1, sample_size):
            ix = v_indices[iPoints[i]]
            jx = v_indices[iPoints[j]]
            
            ratio_1 = abs(k1_c[iPoints[i]] / k1_c[iPoints[j]])
            ratio_2 = abs(k2_c[iPoints[i]] / k2_c[iPoints[j]])
            
            if TH_RATIO_CRV < ratio_1 < 1/TH_RATIO_CRV and TH_RATIO_CRV < ratio_2 < 1/TH_RATIO_CRV:
                reflexNormal = v[ix] - v[jx]
                reflexNormal = reflexNormal / np.linalg.norm(reflexNormal)
                    
                normalBalance = n_vertex[ix] + n_vertex[jx]
                normalBalance = normalBalance / np.linalg.norm(normalBalance)
                                
                if abs(np.dot(normalBalance, reflexNormal)) < 1 - TH_COSINE:
                    if reflexNormal[np.argmax(abs(reflexNormal))] < 0: #Correct direction
                        reflexNormal = -reflexNormal
                    
                    middlePoint = (v[ix] + v[jx]) / 2                    
                    reflexNormal_D = - np.dot(reflexNormal, middlePoint)
                    origin = np.array([0, 0, 0])
                    projection = projectPointPlane(origin, reflexNormal, reflexNormal_D) 
                    
                    #Use projection point instead of D-component
                    xcPair = [ix, jx, reflexNormal, projection, iPoints[i], iPoints[j]]
                    lxPairs.append(xcPair)
                   
    return lxPairs

In [53]:
from datetime import datetime
start = datetime.now()

lxcPairs = baseReflection(c_idx)
len(lxcPairs)

end = datetime.now()
print(end - start)

18107

In [107]:
lfxPairs = np.empty((len(lxcPairs), 2, 3), dtype = float)
for i in range(len(lxcPairs)):
    lfxPairs[i][0] = lxcPairs[i][2]
    lfxPairs[i][1] = lxcPairs[i][3]

In [110]:
lfxPairs[0], lfxPairs[1]

(array([[  -0.5157797 ,    0.83721605,   -0.18177068],
        [ 107.59733783, -174.65250899,   37.91937029]]),
 array([[  0.99570559,  -0.        ,  -0.09257631],
        [-11.57813179,   0.        ,   1.07648359]]))

In [111]:
min(lfxPairs[:,0,0]), min(lfxPairs[:,0,1]), min(lfxPairs[:,0,2]), max(lfxPairs[:,0,0]), max(lfxPairs[:,0,1]), max(lfxPairs[:,0,2])

(-0.7061722639530731,
 -0.7070901969647913,
 -0.70654973450193,
 0.9999875356842178,
 0.9999490190338639,
 0.9999908532248191)

In [112]:
min(lfxPairs[:,1,0]), min(lfxPairs[:,1,1]), min(lfxPairs[:,1,2]), max(lfxPairs[:,1,0]), max(lfxPairs[:,1,1]), max(lfxPairs[:,1,2])

(-127.53843304420224,
 -248.32332630441854,
 -99.17651738190206,
 129.79970582538937,
 6.294460500643349,
 150.03891556453803)

In [123]:
lvxA * iax, lvxA * iay, lvxA * iaz, lvxP * ipx, lvxP * ipy, lvxP * ipz

(-0.8, -0.8, -0.8, -130.4, -260.8, -114.10000000000001)

In [130]:
lvxA * (iax + vax), lvxA * (iay + vay), lvxA * (iaz + vaz), lvxP * (ipx + vpx), lvxP * (ipy + vpy), lvxP * (ipz + vpz)

(1.0, 1.0, 1.0, 130.4, 16.3, 163.0)

In [120]:
lvxA = 0.1
lvxP = round(0.05 * diagonal_bbox, 1)

iax = math.floor(min(lfxPairs[:,0,0]) / lvxA)
iay = math.floor(min(lfxPairs[:,0,1]) / lvxA)
iaz = math.floor(min(lfxPairs[:,0,2]) / lvxA)

ipx = math.floor(min(lfxPairs[:,1,0]) / lvxP)
ipy = math.floor(min(lfxPairs[:,1,1]) / lvxP)
ipz = math.floor(min(lfxPairs[:,1,2]) / lvxP)

iax, iay, iaz, ipx, ipy, ipz

(-8, -8, -8, -8, -16, -7)

In [128]:
min(lfxPairs[:,1,2]), min(lfxPairs[:,1,2]) / lvxP, math.floor(min(lfxPairs[:,1,2]) / lvxP)

(-99.17651738190206, -6.084448919135095, -7)

In [134]:
vax = math.ceil(max(lfxPairs[:,0,0]) / lvxA) - iax + 1 #math.floor(min(lfxPairs[:,0,0]) / lvxA) + 1
vay = math.ceil(max(lfxPairs[:,0,1]) / lvxA) - iay + 1 #math.floor(min(lfxPairs[:,0,1]) / lvxA) + 1
vaz = math.ceil(max(lfxPairs[:,0,2]) / lvxA) - iaz + 1 #math.floor(min(lfxPairs[:,0,2]) / lvxA) + 1

vpx = math.ceil(max(lfxPairs[:,1,0]) / lvxP) - ipx + 1 #math.floor(min(lfxPairs[:,1,0]) / lvxP) + 1
vpy = math.ceil(max(lfxPairs[:,1,1]) / lvxP) - ipy + 1 #math.floor(min(lfxPairs[:,1,1]) / lvxP) + 1
vpz = math.ceil(max(lfxPairs[:,1,2]) / lvxP) - ipz + 1 #math.floor(min(lfxPairs[:,1,2]) / lvxP) + 1

vax, vay, vaz, vpx, vpy, vpz, lvxA, lvxP

(19, 19, 19, 17, 18, 18, 0.1, 16.3)

In [132]:
def locatePoint3d(point, lv, pdx, pdy, pdz):
    ix = math.ceil(point[0] / lv)
    iy = math.ceil(point[1] / lv)
    iz = math.ceil(point[2] / lv)
    
    return ix - pdx, iy - pdy, iz - pdz

In [137]:
start = datetime.now()

v_voxel = np.empty((vax, vay, vaz, ), dtype=object)
p_voxel = np.empty((vax, vay, vaz, ), dtype=object)

for vp in range(1, len(lfxPairs)):
    v_x, v_y, v_z = locatePoint3d(lfxPairs[vp][0], lvxA, iax, iay, iaz)
    v_voxel[v_x,v_y,v_z] = np.append(v_voxel[v_x,v_y,v_z], vp)
    
    p_x, p_y, p_z = locatePoint3d(lfxPairs[vp][1], lvxP, ipx, ipy, ipz)
    p_voxel[p_x,p_y,p_z] = np.append(p_voxel[v_x,v_y,v_z], vp)
    
end = datetime.now()
print(end - start)

0:00:00.591634


In [139]:
v_voxel.shape, p_voxel.shape

((19, 19, 19), (19, 19, 19))

In [138]:
def findneighbors_p(ix, voxel, lvp, pdx, pdy, pdz):
    lx, ly, lz = locatePoint3d(lvp[ix], lvx, pdx, pdy, pdz)
    
    n_voxels = findneighborvoxels(lx, ly, lz, )
    nv_list = []
    
    for nx in range(n_voxels.shape[0]):
        for ny in range(n_voxels.shape[1]):
            for nz in range(n_voxels.shape[2]):
                nv_list.extend(np.delete(voxelg[n_voxels[nx][ny][nz][0], n_voxels[nx][ny][nz][1], n_voxels[nx][ny][nz][2]], 0))
    
    return nv_list

In [ ]:
def findneighborvoxels_p(lx, ly, lz, ):
    neighbors = np.zeros([3,3,3,3], dtype=int)

    for nx in range(3):
        for ny in range(3):
            for nz in range(3):
                neighbors[nx,ny,nz] = [lx + nx - 1, ly + ny - 1, lz + nz - 1]

    if lx == vx - 1:
        neighbors = np.delete(neighbors, 2, axis = 0)
    if lx == 0:
        neighbors = np.delete(neighbors, 0, axis = 0)

    if ly == vy - 1:
        neighbors = np.delete(neighbors, 2, axis = 1)
    if ly == 0:
        neighbors = np.delete(neighbors, 0, axis = 1)

    if lz == vz - 1:
        neighbors = np.delete(neighbors, 2, axis = 2)
    if lz == 0:
        neighbors = np.delete(neighbors, 0, axis = 2)
    
    return neighbors

In [59]:
pair = 22
p_i, p_j = lxcPairs[pair][4], lxcPairs[pair][5]
lxcPairs[pair] #round(k1_c[p_i], 6), round(k2_c[p_i], 6), round(k1_c[p_j], 6), round(k2_c[p_j], 6)

[703,
 165228,
 array([ 0.76075048, -0.64871825, -0.02057528]),
 array([ 84.34912423, -71.92741564,  -2.28130849]),
 14,
 3304]

In [66]:
diagonal_bbox = igl.bounding_box_diagonal(v)
diagonal_bbox

326.35352314012175

In [ ]:
min(lxcPairs[1:,0]), min(v[1:,1]), min(v[1:,2]), max(v[1:,0]), max(v[1:,1]), max(v[1:,2])

In [113]:
def distanceMetricRx(rxPairA, rxPairB):
    module = np.linalg.norm(rxPairA[2]) * np.linalg.norm(rxPairB[2])
    angle = math.acos(min(1.0, abs(np.dot(rxPairA[2], rxPairB[2])) / module))
    
    origin = np.array([0, 0, 0])
    #projectionA = projectPointPlane(origin, rxPairA[2], rxPairA[3])
    #projectionB = projectPointPlane(origin, rxPairB[2], rxPairB[3])
    distance = np.linalg.norm(rxPairA[3] - rxPairB[3])
    
    return distance / diagonal_bbox, angle

In [71]:
#distance / bounding_box, angle
CLUSTER_EP = [0.1, 0.1]

#distances_line / #distance_projections
CLUSTER_RT = [0.05, 0.05]

In [73]:
def clusterRx2(pairs):
    cluster_set = []
    
    for i in range(0, len(pairs)):
        found = False
        c = 0
        
        for c in range(0, len(cluster_set)):
            distance = distanceMetricRx(pairs[i], cluster_set[c][0])
            if distance[0] < CLUSTER_EP[0] and distance[1] < CLUSTER_EP[1]:
                found = True
                break
        
        if not found:
            #mode, original points
            cluster = [pairs[i], [], []]
            cluster_set.append(cluster)
            c = len(cluster_set) - 1
        
        cluster_set[c][1].append(i)
        cluster_set[c][2].append(pairs[i])
    
    return cluster_set

In [84]:
from datetime import datetime
start = datetime.now()

n_cluster2 = clusterRx2(lxcPairs)
len(n_cluster2)

end = datetime.now()
print(end - start)

0:02:41.296660


In [83]:
for x in range(0, len(n_cluster2)):
    if len(n_cluster2[x][1]) > 3:
        print(x, len(n_cluster2[x][1]))

0 29
1 220
2 58
3 177
4 61
5 42
6 213
7 154
8 238
9 165
10 183
11 83
12 155
13 246
14 74
15 72
16 56
17 48
18 79
19 251
20 47
21 132
22 38
23 39
24 33
25 31
26 57
27 19
28 199
29 67
30 50
31 45
32 120
33 26
34 230
35 71
36 68
37 41
38 71
39 59
40 70
41 33
42 44
43 67
44 73
45 62
46 174
47 85
48 74
49 65
50 55
51 63
52 42
53 43
54 47
55 97
56 45
57 43
58 121
59 17
60 31
61 17
62 63
63 210
64 75
65 46
66 63
67 74
68 75
69 65
70 50
71 109
72 36
73 87
74 41
75 42
76 68
77 76
78 40
79 58
80 27
81 41
82 69
83 59
84 79
85 78
86 69
87 29
88 63
89 66
90 53
91 79
92 81
93 49
94 20
96 104
97 27
98 87
99 40
100 49
101 56
102 62
103 45
104 45
105 28
106 87
107 40
108 100
109 62
110 13
111 50
112 48
113 159
114 31
115 37
116 22
117 52
118 22
119 35
120 37
121 207
122 16
123 85
124 49
125 49
126 80
127 20
128 77
129 41
130 21
131 30
132 46
133 30
134 19
135 13
136 47
137 42
138 42
139 36
140 69
141 89
142 94
143 41
144 104
145 14
146 91
147 64
148 47
149 29
150 15
151 32
152 60
153 38
154 31
155 47
1

In [85]:
n_cluster2[13][0]

[[420,
  407139,
  array([0.69496272, 0.00832448, 0.71899759]),
  array([21.21044849,  0.25406548, 21.94399917]),
  8,
  8142],
 [15,
  42,
  62,
  70,
  284,
  445,
  446,
  452,
  713,
  1002,
  1003,
  1075,
  1195,
  1515,
  2164,
  2233,
  2433,
  2464,
  2563,
  2564,
  2565,
  2668,
  2713,
  2732,
  2847,
  2880,
  2889,
  3090,
  3213,
  3214,
  3269,
  3273,
  3304,
  3342,
  3344,
  3423,
  3431,
  3524,
  3575,
  3624,
  3671,
  3673,
  3691,
  3949,
  4017,
  4136,
  4145,
  4217,
  4254,
  4283,
  4307,
  4309,
  4339,
  4478,
  4564,
  4630,
  4636,
  4683,
  4760,
  4766,
  4844,
  5005,
  5048,
  5049,
  5078,
  5079,
  5086,
  5087,
  5137,
  5140,
  5141,
  5169,
  5170,
  5186,
  5196,
  5204,
  5243,
  5423,
  5539,
  5543,
  5544,
  5630,
  5659,
  5660,
  5661,
  5681,
  5779,
  5833,
  5834,
  5835,
  5890,
  5920,
  5953,
  5955,
  5987,
  6116,
  6122,
  6138,
  6148,
  6253,
  6407,
  6737,
  6948,
  7074,
  7075,
  7190,
  7244,
  7350,
  7353,
  7379,
  742

In [17]:
el = igl.edge_lengths(v, f)
el.shape, type(el)

((866694, 3), numpy.ndarray)

In [16]:
area = igl.doublearea(v, f)
area, area.shape

(array([0.05398478, 0.05392532, 0.02727231, ..., 0.06278716, 0.12512584,
        0.0002243 ]),
 (866694,))

In [60]:
bx = igl.bounding_box(v)
bx[0]

array([[  66.1753,    0.    ,  118.817 ],
       [  66.1753,    0.    ,  -16.9141],
       [  66.1753, -259.101 ,  118.817 ],
       [  66.1753, -259.101 ,  -16.9141],
       [ -78.5671,    0.    ,  118.817 ],
       [ -78.5671,    0.    ,  -16.9141],
       [ -78.5671, -259.101 ,  118.817 ],
       [ -78.5671, -259.101 ,  -16.9141]])

In [81]:
np.mean(v[1:,0]), np.mean(v[1:,1]), np.mean(v[1:,2])

(-8.15843497043787, -199.04268463853882, 49.701276046909214)

In [64]:
max(v[:][0]), min(v[:][0]), max(v[:][1]), min(v[:][1]), max(v[:][2]), min(v[:][2])

(0.0, 0.0, 84.6047, -206.716, 84.6832, -206.716)

In [65]:
length_x = max(v[1:,0]) - min(v[1:,0])
length_y = max(v[1:,1]) - min(v[1:,1])
length_z = max(v[1:,2]) - min(v[1:,2])

area = 2 * (length_x * length_y + length_x * length_z + length_y * length_z)
diagonal_bbox = math.sqrt(length_x * length_x + length_y * length_y + length_z * length_z)
diagonal_bbox

326.35352314012175

In [31]:
max(el[:][0]), max(el[:][1]), max(el[:][2]), min(el[:][0]), min(el[:][1]), min(el[:][2])

(0.39736015149987347,
 0.36947404617915947,
 0.36947404617915947,
 0.15156384265385478,
 0.15156384265385478,
 0.07661137317657028)

In [12]:
lvx = round(math.ceil(max(max(el[:][0]), max(el[:][1]), max(el[:][2])) * 200)/100, 2)
lvx

0.8

In [72]:
min(v[1:,0]), min(v[1:,1]), min(v[1:,2]), max(v[1:,0]), max(v[1:,1]), max(v[1:,2])

(-78.5671, -259.101, -16.9141, 66.1753, 0.0, 118.817)

In [78]:
math.floor(min(v[1:,0]) / lvx) * lvx, math.floor(min(v[1:,1]) / lvx) * lvx, math.floor(min(v[1:,2]) / lvx) * lvx

(-79.2, -259.2, -17.6)

In [79]:
math.ceil(max(v[1:,0]) / lvx) * lvx, math.ceil(max(v[1:,1]) / lvx) * lvx, math.ceil(max(v[1:,2]) / lvx) * lvx

(66.4, 0.0, 119.2)

In [13]:
vx = math.ceil(max(v[1:,0]) / lvx) - math.floor(min(v[1:,0]) / lvx) + 1
vy = math.ceil(max(v[1:,1]) / lvx) - math.floor(min(v[1:,1]) / lvx) + 1
vz = math.ceil(max(v[1:,2]) / lvx) - math.floor(min(v[1:,2]) / lvx) + 1

vx, vy, vz

(183, 325, 172)

In [19]:
def locatePoint(point, lv, pdx, pdy, pdz):
    ix = math.ceil(point[0] / lv)
    iy = math.ceil(point[1] / lv)
    iz = math.ceil(point[2] / lv)
    
    return ix - pdx, iy - pdy, iz - pdz

In [21]:
start = datetime.now()

voxelg = np.empty((vx, vy, vz, ), dtype=object)
pdx = math.floor(min(v[1:,0]) / lvx)
pdy = math.floor(min(v[1:,1]) / lvx)
pdz = math.floor(min(v[1:,2]) / lvx)

for vp in range(1, len(v)):
    x, y, z = locatePoint(v[vp], lvx, pdx, pdy, pdz)
    voxelg[x,y,z] = np.append(voxelg[x,y,z], vp)
            
end = datetime.now()
print(end - start)

0:00:07.352878


In [61]:
def findneighbors(ix):
    lx, ly, lz = locatePoint(v[ix], lvx, pdx, pdy, pdz)
    #return np.delete(voxelg[lx,ly,lz], 0), lx, ly, lz
    
    n_voxels = findneighborvoxels(lx, ly, lz)
    nv_list = []
    
    for nx in range(n_voxels.shape[0]):
        for ny in range(n_voxels.shape[1]):
            for nz in range(n_voxels.shape[2]):
                nv_list.extend(np.delete(voxelg[n_voxels[nx][ny][nz][0], n_voxels[nx][ny][nz][1], n_voxels[nx][ny][nz][2]], 0))
    
    return nv_list

In [79]:
x1, y1, z1 = locatePoint(v[12142], lvx, pdx, pdy, pdz)
print(x1, y1, z1, v[12142])

x2, y2, z2 = locatePoint(v[40940], lvx, pdx, pdy, pdz)
print(x2, y2, z2, v[40940])

53 66 133 [ -37.2243 -207.07     88.0148]
54 67 134 [ -36.0692 -205.653    88.8548]


In [26]:
def findneighborvoxels(lx, ly, lz):
    neighbors = np.zeros([3,3,3,3], dtype=int)

    for nx in range(3):
        for ny in range(3):
            for nz in range(3):
                neighbors[nx,ny,nz] = [lx + nx - 1, ly + ny - 1, lz + nz - 1]

    if lx == vx - 1:
        neighbors = np.delete(neighbors, 2, axis = 0)
    if lx == 0:
        neighbors = np.delete(neighbors, 0, axis = 0)

    if ly == vy - 1:
        neighbors = np.delete(neighbors, 2, axis = 1)
    if ly == 0:
        neighbors = np.delete(neighbors, 0, axis = 1)

    if lz == vz - 1:
        neighbors = np.delete(neighbors, 2, axis = 2)
    if lz == 0:
        neighbors = np.delete(neighbors, 0, axis = 2)
    
    return neighbors

In [81]:
ip = 12142
nv = findneighbors(ip)

for nvx in nv:
    if nvx != ip:
        if np.linalg.norm(v[ip] - v[nvx]) < lvx:
            print(np.linalg.norm(v[ip] - v[nvx]), nvx)
        #print(type(v[12142]), v[12142].shape, type(v[nvx]), v[nvx].shape)

0.785706210997468 11805
0.5642921672325477 11809
0.5124028590864881 11813
0.7093535366796989 12133
0.35462827016470233 12136
0.7649322192194545 12255
0.4415997395832699 12256
0.5440334548536628 12259
0.5605798872596043 28
0.5433325685066054 29
0.4247915841915865 11846
0.4021582400001261 11849
0.45341911075735614 11855
0.3545477259833849 30
0.4159405366155017 31
0.5576690775002593 32
0.24047105854967757 12268
0.421997630325106 12275
0.7089248479211308 40764
0.7357440995346142 40855


In [57]:
start = datetime.now()

#a_neighbors = findneighborvoxels(182,324,171) - (183, 325, 172)
a_neighbors = findneighborvoxels(54, 66, 133)
#vx, vy, vz

end = datetime.now()
print(end - start)

0:00:00


In [58]:
a_neighbors.shape[0], a_neighbors, voxelg[a_neighbors[0][1][1][0], a_neighbors[0][1][1][1], a_neighbors[0][1][1][2]]

(3,
 array([[[[ 53,  65, 132],
          [ 53,  65, 133],
          [ 53,  65, 134]],
 
         [[ 53,  66, 132],
          [ 53,  66, 133],
          [ 53,  66, 134]],
 
         [[ 53,  67, 132],
          [ 53,  67, 133],
          [ 53,  67, 134]]],
 
 
        [[[ 54,  65, 132],
          [ 54,  65, 133],
          [ 54,  65, 134]],
 
         [[ 54,  66, 132],
          [ 54,  66, 133],
          [ 54,  66, 134]],
 
         [[ 54,  67, 132],
          [ 54,  67, 133],
          [ 54,  67, 134]]],
 
 
        [[[ 55,  65, 132],
          [ 55,  65, 133],
          [ 55,  65, 134]],
 
         [[ 55,  66, 132],
          [ 55,  66, 133],
          [ 55,  66, 134]],
 
         [[ 55,  67, 132],
          [ 55,  67, 133],
          [ 55,  67, 134]]]]),
 array([None, 30, 31, 32, 12142, 12268, 12275], dtype=object))

In [53]:
voxelg.shape

(183, 325, 172)

In [37]:
alst = igl.adjacency_list(f)
len(alst)

508962

In [67]:
VF,VFi = igl.vertex_triangle_adjacency(f, len(v))
ltf = []
lvt = []

ivx = 1
lvt.append(ivx)
print("*", ivx, alst[ivx])

for fx in VF[VFi[ivx]:VFi[ivx+1]]:
    print(fx, f[fx])
    ltf.append(fx)
            
for vr1 in alst[ivx]:
    if vr1 not in lvt:
        lvt.append(vr1)
        
    print("*", vr1, alst[vr1])
    
    for fx in VF[VFi[vr1]:VFi[vr1+1]]:
        print(fx, f[fx])
        if fx not in ltf:
            ltf.append(fx)
    
    for vr2 in alst[vr1]:
        if vr2 not in lvt:
            lvt.append(vr2)
        
print(sorted(ltf), len(ltf))
print(lvt, len(lvt)) #sorted

* 1 [2, 2717, 2758, 31696, 408030, 448939]
4610 [2717 2758    1]
4628 [   2    1 2758]
52905 [    1     2 31696]
797446 [     1 408030   2717]
845417 [408030      1 448939]
845418 [ 31696 448939      1]
* 2 [1, 3, 2758, 2766, 31696, 31699]
4627 [2758 2766    2]
4628 [   2    1 2758]
4630 [   3    2 2766]
52905 [    1     2 31696]
52912 [31699 31696     2]
52914 [    2     3 31699]
* 2717 [1, 2714, 2716, 2758, 408029, 408030]
4522 [2716 2717 2714]
4601 [2758 2717 2716]
4610 [2717 2758    1]
797440 [  2714   2717 408029]
797441 [408029   2717 408030]
797446 [     1 408030   2717]
* 2758 [1, 2, 2716, 2717, 2759, 2766]
4601 [2758 2717 2716]
4602 [2758 2716 2759]
4610 [2717 2758    1]
4613 [2759 2766 2758]
4627 [2758 2766    2]
4628 [   2    1 2758]
* 31696 [1, 2, 31699, 31704, 448939, 448943]
52905 [    1     2 31696]
52912 [31699 31696     2]
52927 [31696 31699 31704]
765984 [448939  31696 448943]
765988 [ 31704 448943  31696]
845418 [ 31696 448939      1]
* 408030 [1, 2717, 408029, 44893

In [78]:
vx_dtnr = {}

for ix, vtx in enumerate(lvt):
    vx_dtnr[vtx] = ix
    
vx_dtnr

{1: 0,
 2: 1,
 3: 2,
 2758: 3,
 2766: 4,
 31696: 5,
 31699: 6,
 2717: 7,
 2714: 8,
 2716: 9,
 408029: 10,
 408030: 11,
 2759: 12,
 31704: 13,
 448939: 14,
 448943: 15,
 448934: 16,
 448941: 17}

In [104]:
vc = np.array([v[lvt[i]] for i in range(len(lvt))])
vc

array([[ -41.1933, -206.716 ,   84.6047],
       [ -41.1203, -206.716 ,   84.6832],
       [ -40.8555, -206.716 ,   84.9589],
       [ -41.1535, -207.07  ,   84.6047],
       [ -41.1203, -207.07  ,   84.6398],
       [ -41.1203, -206.362 ,   84.7309],
       [ -40.9028, -206.362 ,   84.9589],
       [ -41.4744, -207.07  ,   84.2684],
       [ -41.4913, -207.07  ,   84.2505],
       [ -41.4744, -207.248 ,   84.2505],
       [ -41.5268, -206.716 ,   84.2505],
       [ -41.4744, -206.716 ,   84.3066],
       [ -41.1203, -207.37  ,   84.6047],
       [ -40.9477, -206.007 ,   84.9589],
       [ -41.2371, -206.362 ,   84.6047],
       [ -41.1203, -206.007 ,   84.7782],
       [ -41.4744, -206.362 ,   84.3503],
       [ -41.282 , -206.007 ,   84.6047]])

In [99]:
fc = np.array([[vx_dtnr[f[ltf[i]][0]], vx_dtnr[f[ltf[i]][1]], vx_dtnr[f[ltf[i]][2]]] for i in range(len(ltf))], dtype=int)
fc

array([[ 7,  3,  0],
       [ 1,  0,  3],
       [ 0,  1,  5],
       [ 0, 11,  7],
       [11,  0, 14],
       [ 5, 14,  0],
       [ 3,  4,  1],
       [ 2,  1,  4],
       [ 6,  5,  1],
       [ 1,  2,  6],
       [ 9,  7,  8],
       [ 3,  7,  9],
       [ 8,  7, 10],
       [10,  7, 11],
       [ 3,  9, 12],
       [12,  4,  3],
       [ 5,  6, 13],
       [14,  5, 15],
       [13, 15,  5],
       [10, 11, 16],
       [14, 16, 11],
       [16, 14, 17],
       [15, 17, 14]])

In [105]:
#v1, v2: principal directions | k1, k2: principal values
v1c, v2c, k1c, k2c = igl.principal_curvature(vc, fc)

In [108]:
k1c[0], k2c[0]

(-0.022290373633807904, 0.015499039128917109)

In [81]:
print(ltf, len(ltf))

[4610, 4628, 52905, 797446, 845417, 845418, 4627, 4630, 52912, 52914, 4522, 4601, 797440, 797441, 4602, 4613, 52927, 765984, 765988, 765946, 765971, 765975, 765985] 23


In [6]:
iniv = 1
v_adj = [iniv]
v_adj.extend(alst[iniv])
v_adj

[1, 2, 2717, 2758, 31696, 408030, 448939]

In [60]:
len(f) * 3, len(v), VF.shape, VFi.shape

(2600082, 508963, (2600082,), (508964,))

In [9]:
#from datetime import datetime
start = datetime.now()

iniv = 1
in_face = []

for face in f:
    if iniv in face:
        in_face.append(face)
        
in_face

end = datetime.now()
print(end - start)

0:00:07.044185


In [20]:
start = datetime.now()
adm = igl.adjacency_matrix(f)
type(adm)

end = datetime.now()
print(end - start)

0:00:00.270832


In [32]:
np.percentile(el[:][0], 15), np.percentile(el[:][0], 40), np.percentile(el[:][0], 65), np.percentile(el[:][0], 90)

(0.21320438437691538,
 0.3159386205820164,
 0.3691306659948018,
 0.3892945842127101)

In [33]:
np.percentile(el[:][1], 15), np.percentile(el[:][1], 40), np.percentile(el[:][1], 65), np.percentile(el[:][1], 90)

(0.21396439142398677,
 0.3179653060408734,
 0.3625381841750875,
 0.36749237132085316)

In [34]:
np.percentile(el[:][2], 15), np.percentile(el[:][2], 40), np.percentile(el[:][2], 65), np.percentile(el[:][2], 90)

(0.1606324485921203,
 0.30066757428470375,
 0.3605193510469638,
 0.366915561855675)

In [30]:
el_length = igl.avg_edge_length(v, f)
el_length

0.35079126629804264

In [25]:
p = (el[0][0] + el[0][1] + el[0][2])/2
st = math.sqrt(p*(p-el[0][0])*(p-el[0][1])*(p-el[0][2]))
st * 2

0.053984779198223955

In [20]:
np.linalg.norm(v[f[0][0]] - v[f[0][1]]), np.linalg.norm(v[f[0][0]] - v[f[0][2]]), np.linalg.norm(v[f[0][1]] - v[f[0][2]])

(0.39736015149987347, 0.15156384265385478, 0.3570323150640568)

In [17]:
f[0], v[f[0][0]], v[f[0][1]], v[f[0][2]]

(array([149, 148, 150], dtype=int32),
 array([   3.15213, -256.428  ,   -1.815  ]),
 array([   2.79795, -256.301  ,   -1.94276]),
 array([   3.15213, -256.301  ,   -1.89772]),
 array([0.35703232, 0.15156384, 0.39736015]))

In [82]:
vm = v[1:]
fm = f - [1, 1, 1]
fm

array([[ -41.1933  , -206.716   ,   84.6047  ],
       [ -41.1203  , -206.716   ,   84.6832  ],
       [ -40.8555  , -206.716   ,   84.9589  ],
       ...,
       [  -1.09801 , -160.777   ,  106.774   ],
       [  -1.09701 , -160.763   ,  106.703   ],
       [  -0.524086, -160.673   ,  106.38    ]])